## Import

In [1]:
from numpy import load
import numpy as np
import time 

## Load Data

In [2]:
ag2l = load('/Users/krol/research/Summer2022/Inversion Benchmark Files/ag1000g_2L_gambiae_full.npz')
counts = ag2l['allele_counts']
pos = ag2l['positions']
mask = ag2l['mask']

In [3]:
#create a csv of values so I can manually check that these values are correct that Im getting out of the array
for key, value in ag2l.items():
    np.savetxt("test" + key + ".csv", value, fmt='%f')

## Looking at the Lovely Data

In [4]:
print("Allele_Counts ", counts, counts.shape)
print("Positions ", pos, pos.shape)
print("Mask ", mask, mask.shape)

Allele_Counts  [[0. 1. 1. ... 1. 1. 0.]
 [0. 2. 2. ... 0. 0. 1.]
 [0. 2. 2. ... 0. 0. 0.]
 ...
 [0. 2. 2. ... 0. 0. 0.]
 [0. 2. 2. ... 0. 0. 0.]
 [0. 2. 2. ... 1. 1. 0.]] (81, 596189)
Positions  [   44889    50387   157120 ... 49356239 49356248 49356406] (596189,)
Mask  [0. 0. 0. ... 0. 0. 0.] (596189,)


## Sliding Window

In [5]:
def slide_(allele_counts, windowsize, stepsize):
    #zz = np.zeros(windowsize[0], windowsize[1])
    alpos = []
    for x in range(0, allele_counts.shape[0], stepsize):
        for y in range(0, allele_counts.shape[1], stepsize):
            alcounts = allele_counts[x:x + windowsize[1], y:y+ windowsize[0]]
            if y % windowsize[0] == 0:
                alpos = (pos[y:y + windowsize[0]])
                almask = (mask[y:y + windowsize[0]])
            yield (alcounts, (alpos), (almask))

## Getting the updated Allele_counts, Positions, and Masks

In [6]:
# Creatings a windows/kernel size
winsize = [3,3]

In [7]:
aa = slide_(counts, np.array([winsize[0], winsize[1]]), 1)
bb = slide_(counts, np.array([winsize[0], winsize[1]]), 1)
cc = slide_(counts, np.array([winsize[0], winsize[1]]), 1)

In [8]:
alc_ = []
alp_ = []
alm_ = []

### Getting the masks
We are grabbing the masks based on the kernel size and where the slide_ method left on getting through the allele_count matrix's

In [9]:
%%time
for _, _, almask in bb:
    #print("Print Orginal: ", almask)
    #print("Print modified: ", np.tile(almask, (1,3)))
    alm_.append(np.tile(almask, (1,winsize[0])))

#o = np.unique(temp, axis=0)
#for i in o:
#    alp_.append(np.tile([i], (3,1)))

CPU times: user 2min 5s, sys: 2.46 s, total: 2min 7s
Wall time: 2min 9s


### Getting the Positions 
We are creating matrix's based on the kernel size of the positions that we are grabbing the windows from the allele_count matrix

In [ ]:
%%time
counter = 0
for _, alposition, _ in aa:
    counter = counter + 1
    if counter % 3 == 0:
        alp_.append(np.tile(alposition, (winsize[0], 1)))

### Getting the Counts
Here we are simply appending the array which is our window of elements that we have gathered into a list 

In [ ]:
%%time
for c,_,_ in cc:
    alc_.append(c)

### Cells used for Debugging

In [ ]:
print(len(alp_))
print(len(alc_))
print(len(alm_))

In [ ]:
print(alc_[0:100])
print(counts[0:100])

In [ ]:
print(alm_[0:100])
print(mask[0:100])

In [ ]:
print(alp_[0:100])
print(pos[0:100])

## Saving the to a compressed Numpy file

please don't eat my ram =(

In [ ]:
np.savez_compressed('test', allele_counts=alc_, position=alp_, mask=alm_)